### Imports

In [28]:
# pytorch library
import torch

import torch.nn as nn
# optim is where we get the stochastic gradient descent optimiser from
import torch.optim as optim
# transforms perform a set of operations to an individual item (image) so that it is usable by the model
import torchvision.transforms as transforms
# Tools for creating datasets from folders
from torchvision import datasets
# default_loader enables basic loading of images to be used in the CustomDataset class
from torchvision.datasets.folder import default_loader
# DataLoader creates batches of data in a format the model can use
# Dataset is a general class that allows the retrieval of individual pieces of data via the DataLoader
# All classes that inherit from Dataset must implement the __getitem__ and __len__ 
from torch.utils.data import DataLoader, Dataset
import pandas as pd

from sklearn.metrics import accuracy_score, f1_score

# tensorboard
from torch.utils.tensorboard import SummaryWriter
# We use the current time info for naming logs
from datetime import datetime


### Config

In [29]:
folder = 'tensorboard_initial'
run_name = 'first_experiment'

log_dir = f'runs/{folder}/{run_name}_{datetime.now().strftime("%Y-%m-%d_%H_%M_%S")}'

writer = SummaryWriter(log_dir=log_dir)
num_epochs=15

### Setup transform and define CustomDataset class for iterable datasets

In [30]:
transform=transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Use the train.csv and val.csv csv files to create two datasets
class CustomDataset(Dataset):
    # csv_file is a file path, root_dir is the root of the dataset, transform is an optional instantiation of transforms.Compose
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    # How many items are in the dataset
    def __len__(self):
        return len(self.data)
    
    # Get a specific item from the dataset by index
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx,0] # column 0 contains file paths
        img_path = f"{self.root_dir}/{img_name}"
        # A basic image loader that retruns a Python Imaging Library (PIL) object
        image = default_loader(img_path)
        label = self.data.iloc[idx,1] # column 1 contains image class

        # If there is a transform function, transform the image
        if self.transform:
            image = self.transform(image)

        return image, label

### Setup data

In [31]:
data_path = "../../datasets/bean-leaf-lesions/"

# CSV file paths
train_csv = data_path + 'train.csv'
val_csv = data_path + 'val.csv'

# Create custom datasets
train_dataset = CustomDataset(csv_file=train_csv, root_dir=data_path, transform=transform)
val_dataset = CustomDataset(csv_file=val_csv, root_dir=data_path, transform=transform)

# Create Dataloaders
batch_size=32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) 

In [32]:
# print how many batches are in the train and val dataloaders
print(f"Train batches: {len(train_dataloader)}")
print(f"Val batches: {len(val_data_loader)}")


Train batches: 33
Val batches: 5


### Create NN

In [33]:
# nn.Module is the base class for all NN modules in pytorch
class CNN(nn.Module):
    def __init__(self):
        # CNN class inherits from nn.Module
        # Inheriting allows the use of functions from the parent class
        super(CNN, self).__init__()
        # Input = RGB, Output = number of filters/kernels in the layer, 
        # kernel_size = height&width of convolutional window, padding = number of 0 pixels added to edge of image
        self.conv1 = nn.Conv2d(in_channels=3, out_channels= 16, kernel_size= 3, padding=1)
        # kernel_size = size of the max pooling window
        # step_size determines how much the kernel moves by after each pooling operation
        # With both of these being 2, the spatial dimension is halved
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # The number of outputs increases with the depth of the model because the complexity of the
        # learned representations requires more neurons
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # 32 is the previous layers output size, and 8 is the size of the image after the pooling layer
        # so 32 * 8 * 8 is the number of dimensions in the flattened tensor
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 3) # 3 possible classes in the dataset

    def forward(self, x):
        # Apply conv1 to the input, apply relu activation, apply max pooling
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        # x.view is a pytorch fn used to reshape tensors
        # This flattens the tensor into a 1D vector of 32*8*8 elements
        # The -1 is a placeholder to allow pytorch to automatically compute the size of that dimension
        # The total number of elements in the tensor is used (e.g. tensor contains 100 elems, x.view(5, -1) would resolve -1 to 20)
        x = x.view(-1, 32 * 32 * 32)
        # Puth the flattened tensor through a fully connected layer
        x = torch.relu(self.fc1(x))
        # Resolve to one of the 3 possible classes
        x = self.fc2(x)
        return x

model = CNN()

### Define loss function and optimiser

In [34]:
criterion = nn.CrossEntropyLoss() # Computes softmax as well as loss

# Momentum at 0.9 means that the gradient will be 90% influenced by previous gradients and 10% by the current
# Allows for faster convergence by moving more in steeper situations and less in shallow
optimiser = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

### Train

In [35]:
for epoch in range(num_epochs):
    running_loss=0.0
    total_predictions = []
    true_labels = []
    # Generates tuples containing index 'i' and elements 'data'
    # the 0 argument sets the index to begin at 0
    # the index refers to which batch of data we are currently looking at
    # the data refers to the collection of data points in that batch, with inputs and labels
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data

        # clears any previously generated gradients
        # For each data point, we n
        optimiser.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # computes gradients of the loss wrt model params with back-prop
        loss.backward()
        # Updates model parameters using gradients from loss.backward(), which stored the gradients within the model params
        # Since the model params are stored in the optimiser anyway, we do no need to explicitly handle this arg
        optimiser.step()

        # Record predictions and labels for computing metrics
        # outputs, 1 means that we record the maximum value along the 1st dimension of the outputs tensor
        # the _ is because it returns the value of the item as well as the index, and we do not need the value 
        # (we only need the predicted class)
        _, predicted = torch.max(outputs, 1)
        # we convert predicted to a list for consistency reasons. 
        # It is not strictly necessary in this case but is good practice
        total_predictions.extend(predicted.tolist())
        true_labels.extend(labels.tolist())

        running_loss += loss.item()
    
    # calculate accuracy and F1 score after each epoch
    train_accuracy = accuracy_score(true_labels, total_predictions)
    # type of average can be chosen as appropriate
    # With macro, each class contributes equally to the final metric regardless of class imbalance
    # F1 score is computed for each class independently and then the average is taken
    # This choice might not be appropriate in imbalanced datasets
    train_f1_score = f1_score(true_labels, total_predictions, average='macro')

    # Log metrics using SummaryWriter
    writer.add_scalar('Loss/train', running_loss / len(train_dataloader), epoch+1)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch+1)
    writer.add_scalar('F1_Score/train', train_f1_score, epoch+1)

    # Print the metrics after each epoch
    # Loss is running_loss/len(train_dataloader) because the running loss is the sum of the loss over all the batches
    # Dividing it by the number of batches gives us the average loss
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_dataloader)}, Training Accuracy: {train_accuracy}, Training F1 Score: {train_f1_score}')

Epoch 1, Loss: 1.0694806503527092, Training Accuracy: 0.42649903288201163, Training F1 Score: 0.4248355943542979
Epoch 2, Loss: 0.9725504156314966, Training Accuracy: 0.5512572533849129, Training F1 Score: 0.548715985856119
Epoch 3, Loss: 0.8445745309193929, Training Accuracy: 0.6353965183752418, Training F1 Score: 0.634138632992825
Epoch 4, Loss: 0.7490016944480665, Training Accuracy: 0.6847195357833655, Training F1 Score: 0.6828214958092552
Epoch 5, Loss: 0.6847857345234264, Training Accuracy: 0.7059961315280464, Training F1 Score: 0.7026922078169662
Epoch 6, Loss: 0.6304027718124967, Training Accuracy: 0.7398452611218569, Training F1 Score: 0.7384552511579551
Epoch 7, Loss: 0.6277282373471693, Training Accuracy: 0.7282398452611218, Training F1 Score: 0.725942990656967
Epoch 8, Loss: 0.6032537845048037, Training Accuracy: 0.7504835589941973, Training F1 Score: 0.7482576810433499
Epoch 9, Loss: 0.5874897864731875, Training Accuracy: 0.7572533849129593, Training F1 Score: 0.75703335992

In [36]:
%load_ext tensorboard
%tensorboard --logdir=runs


Reusing TensorBoard on port 6007 (pid 6401), started 2:24:25 ago. (Use '!kill 6401' to kill it.)